In [1]:
import os
from PIL import Image
import matplotlib.pyplot as plt

# 输入危险品名称，返回其在数组的序号
def get_class_index(my_class):
    classes = {'Folding_Knife' : 0, 'Multi-tool_Knife' : 1, 'Scissor' : 2, 'Straight_Knife' : 3, 'Utility_Knife' : 4}
    return classes.get(my_class)

if not os.path.exists('/dataset/OPIXray_Crop/'):
    os.mkdir('/dataset/OPIXray_Crop/')
if not os.path.exists('/dataset/OPIXray_Crop/train/'):
    os.mkdir('/dataset/OPIXray_Crop/train/')
if not os.path.exists('/dataset/OPIXray_Crop/test/'):
    os.mkdir('/dataset/OPIXray_Crop/test/')
if not os.path.exists('/dataset/OPIXray_Crop/val/'):
    os.mkdir('/dataset/OPIXray_Crop/val/')
if not os.path.exists('/dataset/OPIXray_Crop/meta/'):
    os.mkdir('/dataset/OPIXray_Crop/meta/')
if not os.path.exists('/dataset/OPIXray_Crop/crop/'):
    os.mkdir('/dataset/OPIXray_Crop/crop/')

files = open('/dataset/OPIXray_Crop/meta/files.txt', 'a')
with open('/dataset/OPIXray/train/train_knife.txt', 'r') as f:
    data = f.readlines()
    for line in data:
        line = line.strip('\n')
        img = Image.open('/dataset/OPIXray/train/train_image/' + line + '.jpg')
        with open('/dataset/OPIXray/train/train_annotation/' + line + '.txt', 'r') as f:
            annotation = f.readline()
            annotation = annotation.strip('\n')
            results = annotation.split(' ')
            img_crop = img.crop((int(results[2]), int(results[3]), int(results[4]), int(results[5])))
            file_path = '/dataset/OPIXray_Crop/crop/' + line + '_crop.jpg'
            img_crop.save(file_path)
            files.write(line + '_crop.jpg' + ' ' + str(get_class_index(results[1])) + '\n')
            
with open('/dataset/OPIXray/test/test_knife.txt', 'r') as f:
    data = f.readlines()
    for line in data:
        line = line.strip('\n')
        img = Image.open('/dataset/OPIXray/test/test_image/' + line + '.jpg')
        with open('/dataset/OPIXray/test/test_annotation/' + line + '.txt', 'r') as f:
            annotation = f.readline()
            annotation = annotation.strip('\n')
            results = annotation.split(' ')
            img_crop = img.crop((int(results[2]), int(results[3]), int(results[4]), int(results[5])))
            file_path = '/dataset/OPIXray_Crop/crop/' + line + '_crop.jpg'
            img_crop.save(file_path)
            files.write(line + '_crop.jpg' + ' ' + str(get_class_index(results[1])) + '\n')
            
files.close()

In [2]:
# 由于源数据集OPIXray中，没有val，因此，需要重新划分。数据量总数为8885，按照train:test:val=8:1:1的比例划分，分别为：7109:888:888，随机划分。
import os
import random

# 读取文件
with open("/dataset/OPIXray_Crop/meta/files.txt", "r") as files:
    data = files.readlines()

# 打乱数据的顺序
random.shuffle(data)

# 将数据分成8:1:1的比例
split_index1 = int(len(data) * 0.8)
split_index2 = int(len(data) * 0.9)

# 将数据写入三个文件
with open("/dataset/OPIXray_Crop/meta/train.txt", "w") as file_train:
    file_train.writelines(data[:split_index1])

with open("/dataset/OPIXray_Crop/meta/val.txt", "w") as file_val:
    file_val.writelines(data[split_index1:split_index2])

with open("/dataset/OPIXray_Crop/meta/test.txt", "w") as file_test:
    file_test.writelines(data[split_index2:])

In [3]:
# 将图片文件移动到对应的文件夹内
import os,shutil

# 移动文件的位置
# shutil.move(origin_file, dst_folder)

# 训练图片文件名
train_filenames = []
with open("/dataset/OPIXray_Crop/meta/train.txt", "r") as file_train:
    for line in file_train:
        train_filenames.append(line.strip())

# 验证图片文件名
val_filenames = []
with open("/dataset/OPIXray_Crop/meta/val.txt", "r") as file_val:
    for line in file_val:
        val_filenames.append(line.strip())

# 测试图片文件名
test_filenames = []
with open("/dataset/OPIXray_Crop/meta/test.txt", "r") as file_test:
    for line in file_test:
        test_filenames.append(line.strip())

# 源文件夹路径
src_folder = "/dataset/OPIXray_Crop/"
# 目标文件夹路径
dst_folder = "/dataset/OPIXray_Crop/"

# 图片文件名
files = os.listdir("/dataset/OPIXray_Crop/crop/")
for filename in files:
    for train_file in train_filenames:
        if filename == train_file.split(' ')[0]:
            shutil.move("/dataset/OPIXray_Crop/crop/"+filename, "/dataset/OPIXray_Crop/train/")
    for test_file in test_filenames:
        if filename == test_file.split(' ')[0]:
            shutil.move("/dataset/OPIXray_Crop/crop/"+filename, "/dataset/OPIXray_Crop/test/")
    for val_file in val_filenames:
        if filename == val_file.split(' ')[0]:
            shutil.move("/dataset/OPIXray_Crop/crop/"+filename, "/dataset/OPIXray_Crop/val/")